In [2]:
import pyworld as pw
import librosa
import numpy as np
import os
import math
import torch
from pymcd.mcd import Calculate_MCD
import audio as Audio
# wav2vecSER Model
from speechbrain.inference.interfaces import foreign_class

In [3]:
ground_truth_path = "/home/johnny9696/Desktop/DATA/dailytalk_resample/data"
#model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/dailytalk"
#model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/Fitted_v19_flowv2_MPE"
model_output_path = "/mnt/hdd_storage/FCtalkeroutput/result/Dailytalk/400000"
sampling_rate = 22050
hop_length = 256
d_list = os.listdir(model_output_path)
print(d_list)

['925', '1632', '1924', '466', '1804', '2515', '1155', '905', '1391', '718', '1311', '2524', '1645', '122', '1471', '1918', '867', '1614', '3', '382', '339', '1306', '1233', '1671', '661', '79', '1440', '2309', '1360', '2382', '2529', '236', '1735', '1848', '1193', '557', '470', '1406', '1257', '1831', '440', '1535', '1942', '1363', '2408', '247', '2514', '1180', '221', '115', '1764', '2145', '2417', '1758', '1403', '172', '472', '2459', '47', '283', '362', '800', '465', '1282', '1905', '1927', '2052', '1944', '2349', '40', '782', '2263', '1622', '1514', '54', '1435', '1850', '1602', '451', '1510', '2131', '20', '119', '2489', '2266', '2438', '2015', '1865', '2149', '1542', '766', '2447', '2304', '1068', '1815', '2016', '2273', '1873', '1947', '404', '759', '2406', '622', '712', '1485', '1953', '1625', '2497', '1854', '1102', '1774', '1516', '2400', '1780', '962', '749', '1553', '1190', '295', '1368', '855', '24', '2347', '1531', '168', '896', '280', '1394', '1108', '189', '856', '1750

In [4]:
def get_pitch(wav, sampling_rate = 22050, hop_length = 256):
    pitch, t = pw.dio(
        wav.astype(np.float64),
        sampling_rate,
        frame_period=hop_length / sampling_rate * 1000,
    )
    pitch = pw.stonemask(wav.astype(np.float64), pitch, t, sampling_rate)
    return pitch

In [5]:
def get_energy(wav):
# Compute mel-scale spectrogram and energy
        STFT = Audio.stft.TacotronSTFT(
            1024,
            256,
            1024,
            80,
            22050,
            0,
            8000,
        )
        _, energy = Audio.tools.get_mel_from_wav(wav, STFT)
        return energy
        

In [6]:
count = 0
wav_rmse = 0
f0_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i)
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_p = get_pitch(gt_wav)
        gen_p = get_pitch(gen_wav)
        if len(gt_p) > len(gen_p):
            gap = len(gt_p) - len(gen_p)
            gt_p = gt_p[gap//2:-gap//2]
            gen_p = gen_p[:len(gt_p)]
        else:
            gap = len(gen_p) - len(gt_p)
            gen_p = gen_p[gap//2:-gap//2]
            gt_p = gt_p[:len(gt_p)]
        try:
            gt_log = np.nan_to_num(np.log(gt_p))
            gen_log = np.nan_to_num(np.log(gen_p))
            error = abs(gt_log-gen_log)
            non_inf_counter = 0
            rmse = 0
            for data in error:
                if data != np.inf and data != -np.inf and data>= -10 and data <=10:
                    non_inf_counter += 1
                    rmse += data ** 2
            rmse = np.sqrt(rmse/non_inf_counter)
            f0_rmse += rmse
            count += 1
            if count%100 == 0:
                print(count, f0_rmse/count)
        except:
            pass

/tmp/ipykernel_182431/3671593932.py:24: RuntimeWarning: divide by zero encountered in log
  gt_log = np.nan_to_num(np.log(gt_p))
/tmp/ipykernel_182431/3671593932.py:25: RuntimeWarning: divide by zero encountered in log
  gen_log = np.nan_to_num(np.log(gen_p))


100 0.2658961653576722
200 0.2870150289077375
300 0.2864429711792225
400 0.2834567216332175
500 0.28545050395960286
600 0.28467967677665224
700 0.28458208459691653
800 0.2827286184511375
900 0.28385097066413056
1000 0.28225884502668924
1100 0.2829633249238295
1200 0.2839483250226523
1300 0.28484046923839057
1400 0.2852755388117444
1500 0.28378207342966283
1600 0.28296229263296807
1700 0.2827688870045572
1800 0.2834997786057117


In [7]:
print(f0_rmse/count)

0.28331581326821015


In [8]:
count = 0
e_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i)
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_p = get_energy(gt_wav)
        gen_p = get_energy(gen_wav)
        if len(gt_p) > len(gen_p):
            gap = len(gt_p) - len(gen_p)
            gt_p = gt_p[gap//2:-gap//2]
            gen_p = gen_p[:len(gt_p)]
        else:
            gap = len(gen_p) - len(gt_p)
            gen_p = gen_p[gap//2:-gap//2]
            gt_p = gt_p[:len(gt_p)]
        try:
            gt_log = np.nan_to_num(np.log(gt_p))
            gen_log = np.nan_to_num(np.log(gen_p))
            error = abs(gt_log-gen_log)
            non_inf_counter = 0
            rmse = 0
            for data in error:
                if data != np.inf and data != -np.inf and data>= -10 and data <=10:
                    non_inf_counter += 1
                    rmse += data ** 2
            rmse = np.sqrt(rmse/non_inf_counter)
            e_rmse += rmse
            count += 1
            if count%100 == 0:
                print(count, e_rmse/count)

        except:
            pass

100 2.196185207645344
200 2.1023203274443647
300 2.105583265324168
400 2.1052756804779196
500 2.126343371245231
600 2.1122294199925498
700 2.1512656105082058
800 2.1650692535905325
900 2.1802140684437066
1000 2.1810079052097717
1100 2.1940237059304404
1200 2.185749640860426
1300 2.1810806921692705
1400 2.1843413640504883
1500 2.1900886420076153
1600 2.187774498195226
1700 2.193582413338476
1800 2.195826562222065


In [9]:
print(e_rmse/count)

2.1974557296907156


In [10]:
audio_embedder = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
                                       pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")

/home/johnny9696/anaconda3/envs/FS2/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/johnny9696/anaconda3/envs/FS2/lib/python3.8/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sh

In [11]:
w_count = 0
wav_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i)
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_wav_e = torch.tensor(gt_wav)
        gt_wav_emb = audio_embedder.encode_batch(gt_wav_e)
        gt_wav_emb = np.array(gt_wav_emb.squeeze(0))
        gen_wav_e = torch.tensor(gen_wav)
        gen_wav_emb = audio_embedder.encode_batch(gen_wav_e)
        gen_wav_emb = np.array(gen_wav_emb.squeeze(0))
        error = abs(gt_wav_emb - gen_wav_emb)
        rmse = np.sqrt(np.mean(error**2))
        wav_rmse += rmse
        w_count += 1
        if w_count%100 == 0:
            print(w_count, wav_rmse/w_count)

100 0.7523511265218258
200 0.7925460959598422
300 0.815978451197346
400 0.8201917089149355
500 0.8300481767952442
600 0.84527173054715
700 0.8470515032006162
800 0.8432549473736435
900 0.8562097896138827
1000 0.8494314231425524
1100 0.850031384310939
1200 0.8466727622846763
1300 0.8569757173038446
1400 0.8603087292398726
1500 0.8606329367558161
1600 0.8579823982156813
1700 0.8616107411945567
1800 0.8608355258570777


In [12]:
print(wav_rmse/w_count)

0.8607784777069869


In [13]:
mcd_count = 0
mcd = 0
mcd_toolbox = Calculate_MCD(MCD_mode="dtw")
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i)
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        mcd_value = mcd_toolbox.calculate_mcd(os.path.join(t_gt_path, wav_name), os.path.join(t_output_path, wav_name))
        mcd += mcd_value
        mcd_count += 1
        if mcd_count % 100 == 0:
            print(mcd_count, mcd/mcd_count)

100 10.10660770587426
200 10.10888024645096
300 10.011493968878957
400 10.09085413362799
500 10.043320922660788
600 9.96831578403074
700 10.003423483188122
800 9.945544226302438
900 9.890263770035775
1000 9.878555149917473
1100 9.948725505183795
1200 9.929925120825986
1300 9.929186197460687
1400 9.926300914123285
1500 9.956359295617505
1600 9.923667209757577
1700 9.908956793568363
1800 9.914939443629109


In [14]:
print(mcd_count, mcd/mcd_count)

1840 9.907362612219272
